In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!pip install Theano
!pip install keras
!pip install tensorflow

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Convolution2D,MaxPooling2D,Flatten
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


**ANN**

Neuron - input is dependant var output is independant var frst step is calculating sum of dependdant var * weights then applying activation function then prediction

Activation functions

Purpose is to add non-linearity 

x is value of weighted sum
1. Threshold function-> if x>=0 then phi=1 else phi=0
2. Sigmoid function-> phi(x)=1/1+e^-x very useful in predicting output layer as it based on probability
3. Rectifier function-> phi(x)=max(x,0) 
4. Hyperbolic tangent-> phi(x)=1-e^-2x/1+e^-2x 

In hidden layer rectifier func is used while after that sigmoid is used frequently

Most common cost function is 1/2(ypred-y)^2 goal->minimize cost func and this is convex func

Send the cost func again to nn then update the weights and again feed it find cost value again repeat till it got min value

**GRADIENT DESCENT**

It is used to minimize the cost function by going zig-zag throughout.used in convex functions

**STOCHASTIC GRADIENT DESCENT**

We take on row at a time and operate and adjust weights then move to next.It is faster than gradient as it takes row one by one so data is lighter.

**BACK-PROPAGATION**

Errors are backpropagated. 

**WHICH ACTIVATION FUNC TO USE?**

For example, if you want your neural network to predict values that are larger than 1, then tanh or sigmoid are not suitable to be used in the output layer, and we must use ReLU instead.

On the other hand, if we expect the output values to be in the range [0,1] or [-1, 1] then ReLU is not a good choice for the output layer and we must use sigmoid or tanh.

If you perform a classification task and want the neural network to predict a probability distribution over the mutually exclusive class labels, then the softmax activation function should be used in the last layer.

However, regarding the hidden layers, as a rule of thumb, I would strongly suggest you always to use ReLU as an activation for these layers.


In [0]:
dataset=pd.read_csv('/content/drive/My Drive/Churn_Modelling.csv')
X=dataset.iloc[:,3:13].values
y=dataset.iloc[:,13].values

In [0]:
label_encoder_X1 = LabelEncoder()
X[:,1] = label_encoder_X1.fit_transform(X[:,1])
label_encoder_X2 = LabelEncoder()
X[:,2] = label_encoder_X2.fit_transform(X[:,2])

In [0]:
onehot_encoder = ColumnTransformer([('Geography', OneHotEncoder(), [1])], remainder='passthrough')
X = onehot_encoder.fit_transform(X)

In [0]:
# to avoid dummy var trap
X=X[:,1:]

In [0]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [0]:
sc_X=StandardScaler()
X_train=sc_X.fit_transform(X_train)
X_test=sc_X.transform(X_test)

In [0]:
classifier=Sequential()
# adding input and first hidden layer
# more value closer to one of act. func. more activated neuron is i.e more better
classifier.add(Dense(output_dim=6,init='uniform',activation='relu',input_dim=11)) # output_dim is avg of input and output variables i.e 11+1/2
classifier.add(Dense(output_dim=6,init='uniform',activation='relu')) 
#output layer
classifier.add(Dense(output_dim=1,init='uniform',activation='sigmoid'))  # for more classes use softmax and output_dim=3

In [0]:
# compiling ann
classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy']) # loss is lograthmic loss

In [0]:
#fit model
classifier.fit(X_train,y_train,batch_size=10,nb_epoch=100)

In [0]:
y_pred=classifier.predict(X_test)
y_pred=(y_pred>0.5)

In [0]:
accuracy_score(y_test,y_pred)

In [0]:
cm=confusion_matrix(y_test,y_pred)
cm

**CNN**

Convolution is done by feature detector and gives feature map which are relevant features of image.

Convolutional layer is group of feature maps.

Relu Layer-> apply rectifier to conv layer. Motive is to inc non-linearity.
It removes all black values i.e negative val.

In backpropagation,not only weights are adjusted but also feature detectors too.

**WHY LEAKY RELU?**

Another problem we see in ReLU is the Dying ReLU problem where some ReLU Neurons essentially die for all inputs and remain inactive no matter what input is supplied, here no gradient flows and if large number of dead neurons are there in a Neural Network it’s performance is affected, this can be corrected by making use of what is called Leaky ReLU where slope is changed left of x=0 in above figure and thus causing a leak and extending the range of ReLU.

**MAX POOLING**

Select max value from window. It is used to make image invariant to anything.
As max value will remain same.Reducing no. of parameters.Removing more irrelevant info.

**FLATTENING**

After pooling,flatten matrix to one column.As we will, later input to ann.

**FULLY CONNECTED LAYER**

Now adding full ann to flattened and hidden layer should be fully connected.

**SOFTMAX AND CROSS ENTROPY**

Softmax func brings value of output classifier btw 0 and 1 so that prob sum should be one.Cross entropy is cost func which we need to minimize.

Cross->better for classification


In [0]:
classifier=Sequential()

How to improve this cnn?
Add one more conv layer after max pooling

classifier.add(Convolution2D(32,3,3,activation='relu')) 
classifier.add(MaxPooling2D(pool_size=(2,2)))

In [0]:
# step1->convolution
classifier.add(Convolution2D(32,3,3,input_shape=(64,64,3),activation='relu')) # no.of filters,dimn of filter,input_shape,why relu?->no neg values and non-linear behaviour

# pooling
classifier.add(MaxPooling2D(pool_size=(2,2)))

# flattening
classifier.add(Flatten())

# full connection
classifier.add(Dense(output_dim=128,activation='relu'))  # 128 is an experiment.They are hidden nodes
classifier.add(Dense(output_dim=1,activation='sigmoid'))

In [0]:
# compiling the cnn
classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [0]:
# fitting the cnn 
# to prevent overfitting. We need lots of images to get correlation.Need to find patterns.10k not much enough.Create many batches of images and then transforming
# i.e flipping,rotating. These are random. So get more images. Avoid overfitting
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        '/content/drive/My Drive/dataset/training_set',
        target_size=(64,64),
        batch_size=32,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        '/content/drive/My Drive/dataset/test_set',
        target_size=(64,64),
        batch_size=32,
        class_mode='binary')

In [0]:
classifier.fit_generator(
        train_generator,
        steps_per_epoch=8000,
        epochs=25,
        validation_data=validation_generator,
        validation_steps=2000)